In [1]:
import topycal
import os
import glob
import random

INSTR_PATH = os.path.join(os.getcwd(),"afi_txt")

In [2]:
from concurrent.futures import ThreadPoolExecutor, wait, as_completed
def do_fn_on_iter(fn, iterator, num_threads=6):
    futures = []
    if isinstance(num_threads, str):
        num_threads = int(num_threads)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        for elem in iterator:
            futures.append(executor.submit(fn, elem))
    results = []
    for x in as_completed(futures):
        results.append(x.result())
    return results


In [3]:
def get_file_list(limit=500, shuffle=True):
    files = glob.glob("{}/*.txt".format(INSTR_PATH))
    if shuffle:
        random.shuffle(files)
    if limit:
        return files[0:limit]
    else:
        return files
    #data = myfile.read()
    
def read_file(fname):
    with open(fname, errors='replace') as fd:
        return fd.read()

In [4]:
file_list = get_file_list(limit=500)


In [5]:
import re

def load_file(fname):
    with open(fname, 'r') as myfile:
        return (fname,re.sub(r'[\t\n\r\x0b\x0c]',' ',myfile.read()))

def load_corpus(file_list):
    return {f[0]:f[1] for f in do_fn_on_iter(load_file, file_list)}    

In [6]:
corpus_dict = load_corpus(file_list)

In [7]:
import spacy
nlp = spacy.load("en_core_web_md")

In [ ]:
from tqdm import tqdm

class MarketBaskets(object):
    def __init__(self):
        self.lut = {}
    
    def put_in_basket(sentence, doc):
        pass

from collections import defaultdict
import mmh3

MIN_SENT_LEN = 30

def do_market_basket_analysis(doc_dict):
    baskets = defaultdict(set)
    for k,v in tqdm(doc_dict.items()):
        for sent in nlp(v).sents:
            if len(sent) > MIN_SENT_LEN:
                baskets[mmh3.hash64(sent.text.strip())].add(k)
    return baskets

In [ ]:
f = do_market_basket_analysis(corpus_dict)

 26%|██▌       | 129/500 [02:50<08:11,  1.32s/it]

In [ ]:
# trim baskets only 1 long.
MIN_BASKET_LEN = 1
def trim_baskets(baskets):
    return {k:v for k,v in tqdm(baskets.items()) if len(v) > MIN_BASKET_LEN}


In [ ]:
g = trim_baskets(f)

In [ ]:
len(list(g.items()))

In [ ]:
g